In [1]:
# # when in google drive
# from google.colab import drive
# drive.mount('/content/gdrive')
# !pip install scikeras --quiet

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from keras.utils.np_utils import to_categorical 
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization,\
Flatten, LSTM
# from scikeras.wrappers import KerasClassifier
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import load_model

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import OneHotEncoder

NUM_FEATURES = 31
TIMESTEPS = 1
DATASET_LINK = "https://raw.githubusercontent.com/ehandywhyy/ict3204-security-analytics/main/dataset/overall.csv"
TEST_DATA_LINK = "https://raw.githubusercontent.com/ehandywhyy/ict3204-security-analytics/main/dataset/overall_test.csv"

2021-11-27 17:57:43.225210: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-27 17:57:43.225245: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


##### Initialise Seed

In [3]:
# random seed for reproducibility
seed = 10
np.random.seed(seed)

# loading of dataset
df = pd.read_csv(DATASET_LINK)

# # Remove missing values IF AVAILABLE and print head
df = df.dropna()
df.head()

,subject,sessionIndex,rep,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,...,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
0,Andy,1,1,0.142176,0.156880,0.014704,0.127937,0.142299,0.014361,0.155919,...,0.128263,0.085967,-0.042296,0.120442,0.209372,0.088930,0.116911,0.165437,0.048526,0.160258
1,Andy,1,2,0.141793,0.164635,0.022842,0.121625,0.152146,0.030520,0.137039,...,0.144490,0.082397,-0.062093,0.101188,0.126066,0.024878,0.098424,0.130661,0.032238,0.173819
2,Andy,1,3,0.157152,0.147220,-0.009932,0.134796,0.145790,0.010994,0.124892,...,0.135452,0.082423,-0.053029,0.108598,0.155843,0.047244,0.118759,0.146550,0.027791,0.173512
3,Andy,1,4,0.158673,0.142487,-0.016186,0.118908,0.159179,0.040271,0.133175,...,0.114266,0.072556,-0.041711,0.101937,0.145692,0.043755,0.096300,0.122580,0.026280,0.190808
4,Andy,1,5,0.166583,0.157102,-0.009480,0.115094,0.141103,0.026009,0.135121,...,0.125474,0.089247,-0.036227,0.114211,0.149136,0.034925,0.121197,0.230113,0.108917,0.143299


##### Divide dataset into X and Y
##### Normalise features within range 0 (minimum) and 1 (maximum)


In [4]:
dataset = df.values

# divide data into features X and target (Classes) Y
X = dataset[:,3:].astype(float)
Y = dataset[:,0]

# # check for class imbalance
print(df.groupby(Y).size())

Andy       100
Azfar      100
Chris      100
Qikai      100
Safaraz    100
dtype: int64


In [5]:
# convert target Y to one hot encoded Y for model
Y = Y.reshape(-1, 1)
encoder = OneHotEncoder().fit(Y)

# get all the encoded class
print(encoder.get_feature_names_out())

# print X and Y shape
print("X dataset shape: " + str(X.shape))
print("Y dataset shape: " + str(Y.shape))

['x0_Andy' 'x0_Azfar' 'x0_Chris' 'x0_Qikai' 'x0_Safaraz']
X dataset shape: (500, 31)
Y dataset shape: (500, 1)


##### Preparing dataset

In [6]:
# split dataset into train and test of 0.8/0.2 ratio
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=seed)

# normalisation to 0 to 1
scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# reshaping the dataset to include LSTM
X_train = np.asarray(X_train, dtype=np.float32)
X_train = np.reshape(X_train, (X_train.shape[0], TIMESTEPS, X_train.shape[1]))
X_test = np.asarray(X_test, dtype=np.float32)
X_test = np.reshape(X_test, (X_test.shape[0], TIMESTEPS, X_test.shape[1]))

# converting y data to encoding
y_train = encoder.transform(y_train).toarray()
y_test = encoder.transform(y_test).toarray()

num_classes = y_train.shape[1]

print("X train shape: " + str(X_train.shape))
print("Y train shape: " + str(y_train.shape))
print("X test shape: " + str(X_test.shape))
print("Y test shape: " + str(y_test.shape))

X train shape: (400, 1, 31)
Y train shape: (400, 5)
X test shape: (100, 1, 31)
Y test shape: (100, 5)


### Create Model

In [7]:
def create_model():
    # define model
    model = Sequential()
    model.add(LSTM(units=128, return_sequences=True, 
                 input_shape=(TIMESTEPS,NUM_FEATURES)))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(LSTM(units=128, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(LSTM(units=64, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    # Softmax for multi-class classification
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam',
                metrics=['accuracy'])
    return model

##### Wrap Model in KerasClassifier

In [8]:
model = KerasClassifier(build_fn=create_model, epochs=100, 
                            batch_size=10)

### Perform KFold Validation

In [9]:
num_folds = 10
kfold = KFold(n_splits=num_folds, 
              shuffle=True,
              random_state=seed)

##### Get Accuracy from KFold Validation

In [10]:
results = cross_val_score(model, X_train, y_train, 
                          cv=kfold, error_score="raise", verbose=1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
2021-11-27 17:58:11.741783: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: UNKNOWN ERROR (100)
2021-11-27 17:58:11.741831: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-72RQGL3): /proc/driver/nvidia/version does not exist
2021-11-27 17:58:11.742074: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
36/36 [==============================] - 4s 6ms/step - loss: 1.1049 - accuracy: 0.5667
Epoch 2/100
36/36 [==============================] - 0s 6ms/step - loss: 0.5187 - accuracy: 0.8056
Epoch 3/100
36/36 [==============================] - 0s 6ms/step - loss: 0.4272 - accuracy: 0.8500
Epoch 4/100
36/36 [==============================] - 0s 5ms/step - loss: 0.3912 - accuracy: 0.8361
Epoch 5/100
36/36 [==============================] - 0s 5ms/step - loss: 0.2813 - accuracy: 0.8889
Epoch 6/100
36/36 [==============================] - 0s 5ms/step - loss: 0.2965 - accuracy: 0.8861
Epoch 7/100
36/36 [==============================] - 0s 5ms/step - loss: 0.3226 - accuracy: 0.8778
Epoch 8/100
36/36 [==============================] - 0s 5ms/step - loss: 0.2462 - accuracy: 0.9333
Epoch 9/100
36/36 [==============================] - 0s 5ms/step - loss: 0.3725 - accuracy: 0.8722
Epoch 10/100
36/36 [==============================] - 0s 5ms/step - loss: 0.2802 - accuracy: 0.9000
Epoch 11/

36/36 [==============================] - 0s 5ms/step - loss: 0.0970 - accuracy: 0.9722
Epoch 84/100
36/36 [==============================] - 0s 5ms/step - loss: 0.0653 - accuracy: 0.9778
Epoch 85/100
36/36 [==============================] - 0s 5ms/step - loss: 0.0987 - accuracy: 0.9667
Epoch 86/100
36/36 [==============================] - 0s 5ms/step - loss: 0.0679 - accuracy: 0.9778
Epoch 87/100
36/36 [==============================] - 0s 5ms/step - loss: 0.0811 - accuracy: 0.9694
Epoch 88/100
36/36 [==============================] - 0s 5ms/step - loss: 0.0768 - accuracy: 0.9722
Epoch 89/100
36/36 [==============================] - 0s 5ms/step - loss: 0.1135 - accuracy: 0.9583
Epoch 90/100
36/36 [==============================] - 0s 5ms/step - loss: 0.0734 - accuracy: 0.9750
Epoch 91/100
36/36 [==============================] - 0s 6ms/step - loss: 0.0940 - accuracy: 0.9750
Epoch 92/100
36/36 [==============================] - 0s 5ms/step - loss: 0.0786 - accuracy: 0.9806
Epoch 93/100


36/36 [==============================] - 0s 5ms/step - loss: 0.0597 - accuracy: 0.9750
Epoch 65/100
36/36 [==============================] - 0s 7ms/step - loss: 0.0971 - accuracy: 0.9722
Epoch 66/100
36/36 [==============================] - 0s 5ms/step - loss: 0.0988 - accuracy: 0.9667
Epoch 67/100
36/36 [==============================] - 0s 5ms/step - loss: 0.1574 - accuracy: 0.9333
Epoch 68/100
36/36 [==============================] - 0s 5ms/step - loss: 0.0784 - accuracy: 0.9694
Epoch 69/100
36/36 [==============================] - 0s 6ms/step - loss: 0.1225 - accuracy: 0.9639
Epoch 70/100
36/36 [==============================] - 0s 6ms/step - loss: 0.1178 - accuracy: 0.9611
Epoch 71/100
36/36 [==============================] - 0s 7ms/step - loss: 0.0710 - accuracy: 0.9778
Epoch 72/100
36/36 [==============================] - 0s 5ms/step - loss: 0.0619 - accuracy: 0.9833
Epoch 73/100
36/36 [==============================] - 0s 7ms/step - loss: 0.1098 - accuracy: 0.9722
Epoch 74/100


36/36 [==============================] - 0s 5ms/step - loss: 0.1331 - accuracy: 0.9444
Epoch 46/100
36/36 [==============================] - 0s 5ms/step - loss: 0.1125 - accuracy: 0.9639
Epoch 47/100
36/36 [==============================] - 0s 5ms/step - loss: 0.0952 - accuracy: 0.9750
Epoch 48/100
36/36 [==============================] - 0s 5ms/step - loss: 0.1909 - accuracy: 0.9417
Epoch 49/100
36/36 [==============================] - 0s 5ms/step - loss: 0.1036 - accuracy: 0.9667
Epoch 50/100
36/36 [==============================] - 0s 5ms/step - loss: 0.1499 - accuracy: 0.9361
Epoch 51/100
36/36 [==============================] - 0s 5ms/step - loss: 0.1294 - accuracy: 0.9528
Epoch 52/100
36/36 [==============================] - 0s 5ms/step - loss: 0.0935 - accuracy: 0.9722
Epoch 53/100
36/36 [==============================] - 0s 5ms/step - loss: 0.1920 - accuracy: 0.9333
Epoch 54/100
36/36 [==============================] - 0s 5ms/step - loss: 0.1447 - accuracy: 0.9528
Epoch 55/100


...

36/36 [==============================] - 0s 5ms/step - loss: 0.1113 - accuracy: 0.9611
Epoch 33/100
36/36 [==============================] - 0s 6ms/step - loss: 0.2184 - accuracy: 0.9333
Epoch 34/100
36/36 [==============================] - 0s 6ms/step - loss: 0.1234 - accuracy: 0.9583
Epoch 35/100
36/36 [==============================] - 0s 7ms/step - loss: 0.1773 - accuracy: 0.9333
Epoch 36/100
36/36 [==============================] - 0s 6ms/step - loss: 0.1897 - accuracy: 0.9278
Epoch 37/100
36/36 [==============================] - 0s 5ms/step - loss: 0.2251 - accuracy: 0.9389
Epoch 38/100
36/36 [==============================] - 0s 6ms/step - loss: 0.1573 - accuracy: 0.9333
Epoch 39/100
36/36 [==============================] - 0s 5ms/step - loss: 0.1244 - accuracy: 0.9583
Epoch 40/100
36/36 [==============================] - 0s 6ms/step - loss: 0.1911 - accuracy: 0.9389
Epoch 41/100
36/36 [==============================] - 0s 5ms/step - loss: 0.1543 - accuracy: 0.9417
Epoch 42/100


36/36 [==============================] - 0s 6ms/step - loss: 0.2333 - accuracy: 0.9194
Epoch 14/100
36/36 [==============================] - 0s 6ms/step - loss: 0.2104 - accuracy: 0.9306
Epoch 15/100
36/36 [==============================] - 0s 7ms/step - loss: 0.1785 - accuracy: 0.9389
Epoch 16/100
36/36 [==============================] - 0s 8ms/step - loss: 0.2610 - accuracy: 0.9139
Epoch 17/100
36/36 [==============================] - 0s 8ms/step - loss: 0.2546 - accuracy: 0.9139
Epoch 18/100
36/36 [==============================] - 0s 9ms/step - loss: 0.2005 - accuracy: 0.9333
Epoch 19/100
36/36 [==============================] - 0s 8ms/step - loss: 0.2012 - accuracy: 0.9333
Epoch 20/100
36/36 [==============================] - 0s 6ms/step - loss: 0.2105 - accuracy: 0.9361
Epoch 21/100
36/36 [==============================] - 0s 6ms/step - loss: 0.2234 - accuracy: 0.9278
Epoch 22/100
36/36 [==============================] - 0s 6ms/step - loss: 0.2438 - accuracy: 0.9222
Epoch 23/100


36/36 [==============================] - 0s 5ms/step - loss: 0.0960 - accuracy: 0.9667
Epoch 96/100
36/36 [==============================] - 0s 5ms/step - loss: 0.0969 - accuracy: 0.9694
Epoch 97/100
36/36 [==============================] - 0s 5ms/step - loss: 0.0785 - accuracy: 0.9722
Epoch 98/100
36/36 [==============================] - 0s 5ms/step - loss: 0.0877 - accuracy: 0.9694
Epoch 99/100
36/36 [==============================] - 0s 5ms/step - loss: 0.1065 - accuracy: 0.9722
Epoch 100/100
4/4 [==============================] - 1s 3ms/step - loss: 0.0554 - accuracy: 0.9750
Epoch 1/100
36/36 [==============================] - 3s 5ms/step - loss: 1.0325 - accuracy: 0.6000
Epoch 2/100
36/36 [==============================] - 0s 5ms/step - loss: 0.4886 - accuracy: 0.8222
Epoch 3/100
36/36 [==============================] - 0s 6ms/step - loss: 0.3604 - accuracy: 0.8778
Epoch 4/100
36/36 [==============================] - 0s 5ms/step - loss: 0.3291 - accuracy: 0.8611
Epoch 5/100
36/36 

36/36 [==============================] - 0s 5ms/step - loss: 0.0691 - accuracy: 0.9750
Epoch 77/100
36/36 [==============================] - 0s 5ms/step - loss: 0.1021 - accuracy: 0.9667
Epoch 78/100
36/36 [==============================] - 0s 5ms/step - loss: 0.0778 - accuracy: 0.9667
Epoch 79/100
36/36 [==============================] - 0s 5ms/step - loss: 0.1308 - accuracy: 0.9583
Epoch 80/100
36/36 [==============================] - 0s 5ms/step - loss: 0.1285 - accuracy: 0.9694
Epoch 81/100
36/36 [==============================] - 0s 5ms/step - loss: 0.0716 - accuracy: 0.9750
Epoch 82/100
36/36 [==============================] - 0s 5ms/step - loss: 0.0884 - accuracy: 0.9722
Epoch 83/100
36/36 [==============================] - 0s 5ms/step - loss: 0.1523 - accuracy: 0.9667
Epoch 84/100
36/36 [==============================] - 0s 6ms/step - loss: 0.0722 - accuracy: 0.9750
Epoch 85/100
36/36 [==============================] - 0s 10ms/step - loss: 0.1290 - accuracy: 0.9611
Epoch 86/100

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  4.2min finished


##### get validation accuracy

In [11]:
print("Validation Accuracy of %.2f%% (with standard deviation of %.2f%%)" % 
      (results.mean()*100, results.std()*100))

Validation Accuracy of 98.50% (with standard deviation of 1.22%)


##### Fit the Model

In [12]:
# fit the model
es = EarlyStopping(monitor='loss', mode='min', min_delta=0.001, patience=50,
                   verbose=0)
model.fit(X_train, y_train, callbacks=es)

Epoch 1/100
40/40 [==============================] - 3s 5ms/step - loss: 0.9746 - accuracy: 0.6125
Epoch 2/100
40/40 [==============================] - 0s 6ms/step - loss: 0.4530 - accuracy: 0.8225
Epoch 3/100
40/40 [==============================] - 0s 6ms/step - loss: 0.3085 - accuracy: 0.8975
Epoch 4/100
40/40 [==============================] - 0s 5ms/step - loss: 0.3138 - accuracy: 0.8875
Epoch 5/100
40/40 [==============================] - 0s 5ms/step - loss: 0.2803 - accuracy: 0.9000
Epoch 6/100
40/40 [==============================] - 0s 5ms/step - loss: 0.2960 - accuracy: 0.8900
Epoch 7/100
40/40 [==============================] - 0s 5ms/step - loss: 0.2732 - accuracy: 0.8975
Epoch 8/100
40/40 [==============================] - 0s 5ms/step - loss: 0.2228 - accuracy: 0.9225
Epoch 9/100
40/40 [==============================] - 0s 5ms/step - loss: 0.3433 - accuracy: 0.8825
Epoch 10/100
40/40 [==============================] - 0s 5ms/step - loss: 0.2672 - accuracy: 0.8925
Epoch 11/

40/40 [==============================] - 0s 6ms/step - loss: 0.0909 - accuracy: 0.9700
Epoch 84/100
40/40 [==============================] - 0s 6ms/step - loss: 0.0837 - accuracy: 0.9775
Epoch 85/100
40/40 [==============================] - 0s 6ms/step - loss: 0.0985 - accuracy: 0.9800
Epoch 86/100
40/40 [==============================] - 0s 6ms/step - loss: 0.1360 - accuracy: 0.9650
Epoch 87/100
40/40 [==============================] - 0s 6ms/step - loss: 0.1329 - accuracy: 0.9500
Epoch 88/100
40/40 [==============================] - 0s 6ms/step - loss: 0.0963 - accuracy: 0.9775
Epoch 89/100
40/40 [==============================] - 0s 6ms/step - loss: 0.0943 - accuracy: 0.9575
Epoch 90/100
40/40 [==============================] - 0s 6ms/step - loss: 0.0619 - accuracy: 0.9825
Epoch 91/100
40/40 [==============================] - 0s 8ms/step - loss: 0.0949 - accuracy: 0.9700
Epoch 92/100
40/40 [==============================] - 0s 6ms/step - loss: 0.1029 - accuracy: 0.9625
Epoch 93/100


In [13]:
# view model summary
model.model.summary()

# save model
# model.model.save(F"/content/gdrive/My Drive/Colab Notebooks/overall_key_classifier.h5")
model.model.save("model/key_classifier.h5")

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_30 (LSTM)              (None, 1, 128)            81920     
                                                                 
 dropout_30 (Dropout)        (None, 1, 128)            0         
                                                                 
 batch_normalization_30 (Bat  (None, 1, 128)           512       
 chNormalization)                                                
                                                                 
 lstm_31 (LSTM)              (None, 1, 128)            131584    
                                                                 
 dropout_31 (Dropout)        (None, 1, 128)            0         
                                                                 
 batch_normalization_31 (Bat  (None, 1, 128)           512       
 chNormalization)                                    

In [14]:
# load model for Google Drive
# model.model = load_model("/content/gdrive/My Drive/Colab Notebooks/overall_key_classifier.h5")
# model.model = load_model("model/key_classifier.h5")

y_pred = model.predict(X_test)
y_pred = to_categorical(y_pred)

# evaluate predictions
acc = accuracy_score(y_test, y_pred)
print('Testing accuracy: %.3f%%' % acc*100)

# Making the Confusion Matrix
# import sys
# np.set_printoptions(threshold=sys.maxsize)
# cm = confusion_matrix(y_test, y_pred)
# print(cm)

# from sklearn.metrics import roc_curve
# fpr_RF, tpr_RF, thresholds_RF = roc_curve(df.actual_label.values, df.model_RF.values)
# fpr_LR, tpr_LR, thresholds_LR = roc_curve(df.actual_label.values, df.model_LR.values)

Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy: 0.970%Testing accuracy

In [15]:
# import unseen data to check if model works
pred_df = pd.read_csv(TEST_DATA_LINK)
pred_df.head()

,subject,sessionIndex,rep,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,...,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
0,Andy,1,1,0.174239,0.409866,0.235627,0.133840,0.171659,0.037820,0.156984,...,0.137544,0.091721,-0.045824,0.099726,0.099142,-0.000585,0.120912,0.135934,0.015022,0.163508
1,Andy,1,2,0.145289,0.154851,0.009562,0.131217,0.129134,-0.002083,0.130980,...,0.164865,0.111584,-0.053281,0.101935,0.023678,-0.078258,0.140863,0.192325,0.051462,0.156958
2,Andy,1,3,0.122266,0.136321,0.014055,0.114905,0.125308,0.010402,0.144332,...,0.145501,0.095982,-0.049520,0.121049,0.091486,-0.029563,0.111422,0.145751,0.034329,0.169304
3,Andy,1,4,0.121075,0.132562,0.011487,0.134813,0.132049,-0.002763,0.135681,...,0.135072,0.089715,-0.045356,0.117885,0.089209,-0.028676,0.098095,0.139069,0.040974,0.168298
4,Andy,1,5,0.141361,0.124721,-0.016640,0.098268,0.149003,0.050735,0.128713,...,0.121480,0.078897,-0.042583,0.108607,0.049435,-0.059172,0.131348,0.156398,0.025050,0.170112


In [16]:
pred_dataset = pred_df.values

# divide data into features X
# X_new = new_dataset[:,3:].astype(float)

pred_row=pred_df.iloc[:,3:]
# print("check name")
# print(pred_df.iloc[0:7,0:1])

# convert to
pred_row = pred_row.values.tolist()
pred_row = scaler.transform(pred_row)
pred_arr = np.asarray(pred_row, dtype=np.float32)
pred_arr = np.reshape(pred_arr, (pred_row.shape[0], TIMESTEPS, pred_arr.shape[1]))

In [17]:
# get prediction and its label
pred = model.predict(pred_arr)
pred = to_categorical(pred)
pred = encoder.inverse_transform(pred)

pred = np.squeeze(pred)

pred_proba = model.predict_proba(pred_arr)
acc = np.max(pred_proba, axis=1)

print(pred)
print(acc)

['Andy' 'Andy' 'Andy' 'Andy' 'Andy' 'Azfar' 'Azfar' 'Azfar' 'Azfar'
 'Azfar' 'Chris' 'Chris' 'Chris' 'Chris' 'Chris' 'Qikai' 'Qikai' 'Qikai'
 'Qikai' 'Safaraz' 'Safaraz' 'Safaraz' 'Safaraz' 'Safaraz' 'Safaraz']
[0.99962795 0.999111   0.9947094  0.9992982  0.9928271  0.9990816
 0.99991417 0.99989915 0.9998684  0.99996746 0.9669183  0.999388
 0.9995852  0.99847883 0.99749666 0.9969258  0.9966577  0.9672659
 0.97514296 0.85102    0.99662316 0.9987056  0.9985481  0.98481226
 0.9961481 ]
